# computing birth death rates from fortranic trajectories
Tim Tyree<br>
7.7.2022

In [1]:
from lib.my_initialization import *
from lib import *

In [119]:
def comp_death_rates(df_n,n_trgt=6):
    """measure n_trgt --> n_trgt-2 times
    
    Example Usage:
df_rate = comp_death_rates(df_n,n_trgt=6)
    """
    boo_trgt=df_n==n_trgt
    boo = ((df_n.shift(-1)==n_trgt-2)&(boo_trgt))
    boo.any()

    boo_event=df_n.shift(-1).diff()!=0 #for n-2 only (simple)
    event_times = boo_event[boo_event].index.values

    boo = ((df_n.shift(-1)==n_trgt-2)&(boo_trgt))

    #count how many index values appear before each 
    dur_lst=[]
    death_rate_lst=[]
    for tf in boo[boo].index.values:
        ti = np.max(event_times[event_times<tf])
        dur=tf-ti
        dur_lst.append(dur)
        death_rate_lst.append(1/dur)
    df_rate = pd.DataFrame(dict({
        'n_before':n_trgt,
        'dur':dur_lst,
        'death_rate':death_rate_lst
    }))
    return df_rate

In [128]:
#fortranic spiral tip model test case
def routine(trial_num):
    log_folder_parquet='/Users/timothytyree/Documents/GitHub/care/notebooks/Data/from_wjr/tippos_per_001_log/'
    try:
        df=load_parquet_by_trial_num(trial_num=trial_num,folder_parquet=log_folder_parquet)
        df_n=df.groupby(by='t')['n'].mean()
        df_rate = comp_death_rates(df_n,n_trgt=6)
        return df_rate
    except Exception as e:
        return None

In [129]:
task_lst=list(range(640))

In [130]:
npartitions=10
retval=eval_routine_daskbag(routine,task_lst,npartitions,printing=True)
df_rates=pd.concat(retval)
#NOTE: if this finishes in less than ~53 minutes, then it saves time compared to a simple for loop
#it ran in ~15 minutes. not bad.

run time for evaluating routine was 4.55 seconds, yielding 640 values returned


In [131]:
df_rates.mean()

n_before       6.000000
dur           11.683683
death_rate     0.299402
dtype: float64

death_rate for n=6 --> n=4:    0.299402 Hz

Q: Does ^this agree with previously known values?


In [132]:
1/11.68

0.08561643835616438

_from wj's table for LR (A=25):_

No2, rate
2, 0.01820
3, 0.0377


In [133]:
1/0.0182

54.94505494505494

In [134]:
1/0.0377

26.525198938992045

conclusion: these rates are ~0.5X what i get from WJ's data for the LR model when diffCoeff=0.001.

THEREFORE: the annihilation rates in the LR model for Vidmar & Rappel (2019) DO in fact have diffCoeff=0.0005 for the LR model.

i.e. __WJ did not use tip_pos_001 to compute bdrates in Vidmar & Rappel (2019)__